# Distributed training  

### Import packages 
* os -  provides a portable way of using operating system dependent functionality.

In [ ]:
import os

### Importing standard python packages
* utils - a collection of small Python functions and classes which make common patterns shorter and easier.
* numpy - package for scientific computing with Python.

In [ ]:
import utils
import numpy as np

### Importing tensorflow packages
* tensorflow - library for dataflow programming across a range of tasks.
* tensorflow.contrib.learn.python.learn.datasets - contains dataset utilities and synthetic/reference datasets, for getting the mnist dataset

In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib.learn.python.learn.datasets import mnist

### Importing amazon packages
* sagemaker - Python SDK for training and deploying machine learning models on Amazon SageMaker.
* get_execution_role - Return the role ARN whose credentials are used to call the API.
* sagemaker.tensorflow - The Amazon SageMaker custom TensorFlow code. 

In [ ]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.tensorflow import TensorFlow

## Getting and preprocessing the dataset
* Read the mnist dataset
* split it into three : train, validation and test
* instantiate a sagemaker session
* upload our datasets to an S3 location.

In [ ]:
data_sets = mnist.read_data_sets('data', dtype=tf.uint8, reshape=False, validation_size=5000)

In [ ]:
utils.convert_to(data_sets.train, 'train', 'data')
utils.convert_to(data_sets.validation, 'validation', 'data')
utils.convert_to(data_sets.test, 'test', 'data')

In [ ]:
sagemaker_session = sagemaker.Session()

In [ ]:
inputs = sagemaker_session.upload_data(path='data', key_prefix='data/mnist')

## Training the model
* You will need the training script predefined.
* Define a tensorflow estimator object and pass in the python script as the entry point parameter.
* Your TensorFlow training script must be a Python 2.7 source file. It should define the following methods.
* **model_fn**: Defines the model that will be trained.
* **train_input_fn**: Preprocess and load training data.
* **eval_input_fn**: Preprocess and load evaluation data.
* **serving_input_fn**: Defines the features to be passed to the model during prediction.
* Get the role ARN whose credentials are used to call the API.
* To perform distributed training,the instance count is set to 2. 
* Invoke the fit method to train the model. The fit method will create a training job in two ml.c4.xlarge instances. The logs will show the instances doing training, evaluation, and incrementing the number of training steps.
* Invoke the deploy method to create an endpoint.

In [ ]:
!cat 'mnist.py'

In [ ]:
role = get_execution_role()
role

In [ ]:
mnist_estimator = TensorFlow(entry_point='mnist.py',
                             role=role,
                             training_steps=1000, 
                             evaluation_steps=100,
                             train_instance_count=2,
                             train_instance_type='ml.c4.xlarge')

In [ ]:
mnist_estimator.fit(inputs)

In [ ]:
mnist_predictor = mnist_estimator.deploy(initial_instance_count=1,
                                             instance_type='ml.m4.xlarge')

## Validating the model
* get some data for testing
* call the predictor to compare the labels from test data and the predicted labels

In [ ]:
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

In [ ]:
for i in range(10):
    data = mnist.test.images[i].tolist()
    tensor_proto = tf.make_tensor_proto(values=np.asarray(data), shape=[1, len(data)], dtype=tf.float32)
    predict_response = mnist_predictor.predict(tensor_proto)
    
    print("========================================")
    label = np.argmax(mnist.test.labels[i])
    print("label is {}".format(label))
    prediction = predict_response['outputs']['classes']['int64Val'][0]
    print("prediction is {}".format(prediction))

## Delete the endpoint

In [ ]:
sagemaker_session.delete_endpoint(mnist_predictor.endpoint)